In [ ]:
pip install phe


In [ ]:
import pandas as pd
from phe import paillier
from collections import defaultdict
import multiprocessing

dataframe = pd.read_csv(r'/content/primary-2015-election-summary.csv')
sorted_df = dataframe.sort_values(by='Contest Name')


sorted_df.to_csv('sorted_contests.csv', index=False)

In [ ]:
sorted_df

,Votes,Prec. Count,Party,Dist. Type,Dist. Code,Contest Name,Cand. Num,Cand. Name,Vote for,Ref
2103,280,6,REP,NaN,NaN,Auditor BELLEVUE,1,Lenora Z. Bauer,1,NaN
578,29,6,DEM,NaN,NaN,Auditor BELLEVUE,1,WRITE-IN,1,NaN
2104,5,6,REP,NaN,NaN,Auditor BELLEVUE,2,WRITE-IN,1,NaN
579,95,2,DEM,NaN,NaN,Auditor EAST DEER,1,Jennifer L. Savka,1,NaN
602,85,2,DEM,NaN,NaN,Auditor EAST DEER,1,Timothy Daniel McKenna,1,NaN
...,...,...,...,...,...,...,...,...,...,...
577,9,38,DEM,NaN,NaN,Treasurer MT LEBANON,2,WRITE-IN,1,NaN
576,2005,38,DEM,NaN,NaN,Treasurer MT LEBANON,1,Joseph Senko,1,NaN
2102,35,38,REP,NaN,NaN,Treasurer MT LEBANON,1,WRITE-IN,1,NaN
2782,1166,17,NaN,MUN,228.0,Wilkinsburg Question,1,Yes,1,Y


In [ ]:
import pandas as pd
from phe import paillier
from collections import defaultdict
import multiprocessing

# Generate multiple Paillier key pairs with varying sizes
key_sizes = [1024, 2048, 3072]
key_pairs = [paillier.generate_paillier_keypair(n_length=size) for size in key_sizes]

df = sorted_df

def encrypt_votes_with_variations(votes, key_pairs):
    encrypted_votes = []
    for vote in votes:

        encrypted_vote = key_pairs[0][0].encrypt(vote)

        intermediate_value = key_pairs[0][1].decrypt(encrypted_vote)
        encrypted_intermediate = key_pairs[1][0].encrypt(intermediate_value)

        final_value = key_pairs[1][1].decrypt(encrypted_intermediate)
        final_encrypted_vote = key_pairs[2][0].encrypt(final_value)
        encrypted_votes.append(final_encrypted_vote)
    return encrypted_votes


def sum_encrypted_votes(encrypted_votes):
    return sum(encrypted_votes, start=key_pairs[2][0].encrypt(0))


def chunk_data(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]


def parallel_encrypt_votes(votes_chunks, key_pairs):
    with multiprocessing.Pool() as pool:
        encrypted_votes_chunks = pool.starmap(encrypt_votes_with_variations, [(chunk, key_pairs) for chunk in votes_chunks])
    return encrypted_votes_chunks


def parallel_sum_encrypted_votes(encrypted_votes_chunks):
    with multiprocessing.Pool() as pool:
        encrypted_totals_chunks = pool.map(sum_encrypted_votes, encrypted_votes_chunks)
    return encrypted_totals_chunks


results = defaultdict(lambda: defaultdict(list))


grouped = df.groupby('Contest Name')

for contest_name, group in grouped:
    candidates = group['Cand. Num'].unique()
    candidate_names = {row['Cand. Num']: row['Cand. Name'] for idx, row in group.iterrows()}


    votes = group['Vote for'].tolist()


    votes_chunks = list(chunk_data(votes, 1000))
    encrypted_votes_chunks = parallel_encrypt_votes(votes_chunks, key_pairs)


    total_encrypted_votes = parallel_sum_encrypted_votes(encrypted_votes_chunks)


    total_encrypted = sum(total_encrypted_votes, start=key_pairs[2][0].encrypt(0))


    decrypted_total_votes = key_pairs[2][1].decrypt(total_encrypted)

    candidate_vote_counts = defaultdict(int)
    for encrypted_vote_chunk in encrypted_votes_chunks:
        for vote in encrypted_vote_chunk:
            decrypted_round3 = key_pairs[2][1].decrypt(vote)
            encrypted_intermediate = key_pairs[1][0].encrypt(decrypted_round3)
            decrypted_round2 = key_pairs[1][1].decrypt(encrypted_intermediate)
            encrypted_original = key_pairs[0][0].encrypt(decrypted_round2)
            decrypted_original = key_pairs[0][1].decrypt(encrypted_original)
            candidate_vote_counts[decrypted_original] += 1


    for candidate in candidates:
        results[contest_name][candidate_names[candidate]].append(candidate_vote_counts[candidate])


winners = {}
for contest_name, candidates in results.items():
    winner = max(candidates.items(), key=lambda item: item[1][0])
    winners[contest_name] = winner


for contest_name, (candidate_name, votes) in winners.items():
    print(f"The winner of the {contest_name} contest is {candidate_name} with {votes[0]} votes.")


for contest_name, candidates in results.items():
    for candidate, votes in candidates.items():
        encrypted_total_again = key_pairs[2][0].encrypt(votes[0])
        print(f"Encrypted total votes for {candidate} in {contest_name}: {encrypted_total_again.ciphertext()}")


The winner of the Auditor BELLEVUE contest is WRITE-IN with 3 votes.
The winner of the Auditor EAST DEER contest is WRITE-IN with 9 votes.
The winner of the Auditor ETNA contest is WRITE-IN with 0 votes.
The winner of the Auditor FAWN contest is WRITE-IN with 6 votes.
The winner of the Auditor FINDLAY contest is WRITE-IN with 2 votes.
The winner of the Auditor FORWARD contest is WRITE-IN with 6 votes.
The winner of the Auditor FRAZER contest is WRITE-IN with 2 votes.
The winner of the Auditor GLENFIELD contest is WRITE-IN with 6 votes.
The winner of the Auditor HARMAR contest is WRITE-IN with 2 votes.
The winner of the Auditor INDIANA contest is WRITE-IN with 4 votes.
The winner of the Auditor KILBUCK contest is WRITE-IN with 2 votes.
The winner of the Auditor MARSHALL contest is WRITE-IN with 2 votes.
The winner of the Auditor MOON contest is WRITE-IN with 2 votes.
The winner of the Auditor MT OLIVER contest is WRITE-IN with 5 votes.
The winner of the Auditor NORTH FAYETTE contest is 